En primer lugar, importamos tensorflow.

In [1]:
import tensorflow as tf

Importamos el dataser MNIST desde tensorflow, lo dividimos entre training y validation set y normalizamos los valores de los pixeles para que estén entre 0 y 1 para acelerar el entrenamiento.

In [2]:
mnist = tf.keras.datasets.mnist
(training_images, training_labels), (test_images, test_labels) = mnist.load_data()
training_images=training_images.reshape(60000, 28, 28, 1) / 255.0
test_images = test_images.reshape(10000, 28, 28, 1) / 255.0

Definimos un modelo con dos primeras capas convolucionales con 64 filtros de 3x3 y función de activación 'relu'. Luego agregamos una capa MaxPooling a cada una para reducir la dimensión de la red. 

Luego entramos a una red fully-conected con 512 nodos y finalmente una capa fully-conected con 10 nodos donde cada nodo representa un número del 0 al 9 con una función softmax para quedarnos con el resultado con la mayor probabilidad.

In [3]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(64, (3,3), activation=tf.nn.relu, input_shape=(28, 28, 1)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3,3), activation=tf.nn.relu),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation=tf.nn.relu),
    tf.keras.layers.Dense(10, activation=tf.nn.softmax)
])

Creamos un Callback para que la red deje de entrenar cuando consigamos un accuracy mayor al 99% en el validation set.

In [4]:
class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if(logs.get('val_accuracy') > 0.99):
            print("\nAlcanzamos un accuracy mayor al 99% entonces paramos!")
            self.model.stop_training = True

callbacks = myCallback()

Entrenamos a la red con un optimizador Adam y 10 épocas. Gracias al callback la función se detiene luego de solo 3 épocas con una accuracy mayor al 99% en el validation set!

In [5]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
history = model.fit(training_images, 
                    training_labels, 
                    epochs=10, 
                    validation_data = (test_images, test_labels), 
                    callbacks=[callbacks])

Epoch 1/10
1875/1875 [==============================] - 131s 70ms/step - loss: 0.1065 - accuracy: 0.9670 - val_loss: 0.0468 - val_accuracy: 0.9835
Epoch 2/10
1875/1875 [==============================] - 137s 73ms/step - loss: 0.0367 - accuracy: 0.9880 - val_loss: 0.0354 - val_accuracy: 0.9881
Epoch 3/10
1875/1875 [==============================] - ETA: 0s - loss: 0.0244 - accuracy: 0.9922
Alcanzamos un accuracy mayor al 99% entonces paramos!
1875/1875 [==============================] - 129s 69ms/step - loss: 0.0244 - accuracy: 0.9922 - val_loss: 0.0289 - val_accuracy: 0.9910
